## 1.

In [38]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
import random

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

In [39]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [40]:
tfidfvect = TfidfVectorizer()

In [41]:
X_train = tfidfvect.fit_transform(newsgroups_train.data)

In [42]:
y_train = newsgroups_train.target

In [43]:
# Fijar la semilla para reproducibilidad
random.seed(42)

# Seleccionar 5 índices al azar utilizando la semilla establecida
random_indices = random.sample(range(len(newsgroups_train.data)), 5)

# Obtener los vectores correspondientes a los índices seleccionados
reference_vectors = X_train[random_indices]

In [44]:

# Función para imprimir documentos más similares con formato detallado
def print_most_similar_documents(X, y_train, newsgroups_train, random_indices, reference_vectors):
    for i, index in enumerate(random_indices):
        # Calcular similitud coseno del vector de referencia con todos los documentos
        cossim = cosine_similarity(reference_vectors[i], X)[0]

        print(f"******* Documento índice {index} *****")

        # Obtener los 5 documentos más similares (excluyendo el propio documento)
        mostsim = np.argsort(cossim)[::-1][1:6]

        print("Los Documentos más similares (Índices):")
        print(mostsim)

        # Mostrar la clase a la que pertenece el documento de referencia
        class_belong = newsgroups_train.target_names[y_train[index]]
        print(f"Clase a la que pertenece el documento: {class_belong}")

        # Obtener las clases de los documentos similares
        similarity_class = [newsgroups_train.target_names[y_train[mostidx]] for mostidx in mostsim]
        print("Los documentos similares a qué clase pertenecen:")
        print("\\\n".join(similarity_class))

        print("*************************************\n")




In [45]:
# Llamar a la función con tus datos
print_most_similar_documents(X_train, newsgroups_train.target, newsgroups_train, random_indices, reference_vectors)

******* Documento índice 10476 *****
Los Documentos más similares (Índices):
[ 5064  9623 10575 10836  2350]
Clase a la que pertenece el documento: rec.sport.hockey
Los documentos similares a qué clase pertenecen:
rec.sport.hockey\
talk.politics.mideast\
sci.crypt\
alt.atheism\
sci.crypt
*************************************

******* Documento índice 1824 *****
Los Documentos más similares (Índices):
[9921 6364 5509 2641 4359]
Clase a la que pertenece el documento: comp.sys.mac.hardware
Los documentos similares a qué clase pertenecen:
comp.sys.mac.hardware\
comp.sys.mac.hardware\
comp.sys.mac.hardware\
comp.sys.mac.hardware\
comp.sys.mac.hardware
*************************************

******* Documento índice 409 *****
Los Documentos más similares (Índices):
[3444 5799 5905 1764 3364]
Clase a la que pertenece el documento: comp.graphics
Los documentos similares a qué clase pertenecen:
comp.graphics\
comp.graphics\
comp.graphics\
comp.graphics\
comp.graphics
****************************

## Conclusiones del análisis

A partir de los resultados obtenidos, podemos observar cómo la **similitud coseno está detectando correctamente temas relacionados** en la mayoría de los casos. A continuación, se analiza cada documento:

---

### Documento Índice 10476 (`rec.sport.hockey`)
- **Documentos más similares incluyen:**
  - Otros relacionados con **hockey (`rec.sport.hockey`)**.
  - Documentos que no parecen relacionados como `talk.politics.mideast`, `sci.crypt` y `alt.atheism`.
- **Conclusión:**
  Aunque detecta bien documentos de hockey, está capturando documentos irrelevantes posiblemente por palabras comunes o patrones de redacción.

---

### Documento Índice 1824 (`comp.sys.mac.hardware`)
- **Documentos más similares incluyen:**
  - Todos pertenecen a `comp.sys.mac.hardware`.
- **Conclusión:**
  Muy buena precisión, indicando que la similitud coseno funciona bien cuando el tema está claramente definido y específico.

---

### Documento Índice 409 (`comp.graphics`)
- **Documentos más similares incluyen:**
  - Todos pertenecen a `comp.graphics`.
- **Conclusión:**
  Excelente identificación. Los documentos de gráficos probablemente comparten un vocabulario técnico específico que los diferencia bien.

---

### Documento Índice 4506 (`rec.autos`)
- **Documentos más similares incluyen:**
  - Mayormente relacionados con `rec.autos`.
  - También se encuentran relacionados `comp.sys.mac.hardware` y `rec.motorcycles`.
- **Conclusión:**
  Buen desempeño general. La similitud coseno está detectando correctamente temas relacionados a autos y motocicletas (ambos son tópicos automotrices). El único error notable es el documento de hardware de Mac.

---

### Documento Índice 4012 (`rec.sport.hockey`)
- **Documentos más similares incluyen:**
  - Mayormente relacionados con `rec.sport.hockey`.
  - También relacionados con `rec.sport.baseball` (deporte similar).
  - Un documento de `soc.religion.christian`, que no tiene relación.
- **Conclusión:**
  El algoritmo detecta bien temas relacionados con deportes, pero presenta errores ocasionales por coincidencias en palabras comunes.




## 2.

In [47]:
# Vectorización con TfidfVectorizer con parámetros iniciales
tfidfvect = TfidfVectorizer(stop_words='english', max_df=0.8, min_df=5)
X_train = tfidfvect.fit_transform(newsgroups_train.data)
X_test = tfidfvect.transform(newsgroups_test.data)

# Guardar las etiquetas
y_train = newsgroups_train.target
y_test = newsgroups_test.target

# Función para entrenar y evaluar un modelo Naive Bayes
def train_and_evaluate_model(model, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    f1_macro = f1_score(y_test, y_pred, average='macro')
    print(f"{model_name} - F1 Score (Macro): {f1_macro:.4f}")
    return f1_macro

# Entrenar y evaluar MultinomialNB
multinomial_nb = MultinomialNB(alpha=0.5)
f1_multinomial = train_and_evaluate_model(multinomial_nb, "MultinomialNB")

# Entrenar y evaluar ComplementNB
complement_nb = ComplementNB(alpha=0.5)
f1_complement = train_and_evaluate_model(complement_nb, "ComplementNB")


MultinomialNB - F1 Score (Macro): 0.6611
ComplementNB - F1 Score (Macro): 0.6827


## Conclusiones del Desempeño de los Modelos Naive Bayes

### MultinomialNB
- **F1 Score (Macro): 0.6611**
- Este modelo funciona razonablemente bien, pero es más sensible a clases que tienen más documentos, ya que no maneja adecuadamente datos desbalanceados.

### ComplementNB
- **F1 Score (Macro): 0.6827**
- Este modelo mejora el rendimiento general en comparación con `MultinomialNB`.
- Está diseñado específicamente para ser más robusto con clases desbalanceadas, lo cual explica la mejora en el F1 Score.

---

### Comparación General
- `ComplementNB` supera consistentemente a `MultinomialNB` con un puntaje F1 Macro más alto (0.6827 vs. 0.6611).
- La diferencia no es enorme, pero es significativa considerando que estamos utilizando un modelo sencillo y un preprocesamiento estándar.



## 3.

In [49]:
# Transponer la matriz documento-término para obtener término-documento
X_train_transposed = X_train.T  # Matriz Término-Documento

# Obtener el vocabulario mapeado a índices
vocab = tfidfvect.get_feature_names_out()

# Selección manual de palabras relevantes
selected_words = ['hockey', 'computer', 'religion', 'space', 'science']

# Función para encontrar palabras similares usando similitud coseno
def find_similar_words(X_transposed, vocab, words):
    for word in words:
        if word in vocab:
            # Obtener el índice de la palabra
            word_index = np.where(vocab == word)[0][0]

            # Calcular la similitud coseno entre la palabra y todas las demás
            word_vector = X_transposed[word_index]
            cossim = cosine_similarity(word_vector, X_transposed)[0]

            # Encontrar los 5 términos más similares
            most_similar_indices = np.argsort(cossim)[::-1][1:6]  # Excluyendo la propia palabra
            most_similar_words = [(vocab[i], cossim[i]) for i in most_similar_indices]

            # Mostrar resultados
            print(f"\n******* Palabra: {word} *******")
            for similar_word, score in most_similar_words:
                print(f"{similar_word} - Similitud: {score:.4f}")
            print("***********************************")
        else:
            print(f"\nLa palabra '{word}' no está en el vocabulario.\n")

# Llamar a la función para mostrar palabras similares
find_similar_words(X_train_transposed, vocab, selected_words)



******* Palabra: hockey *******
ncaa - Similitud: 0.2672
nhl - Similitud: 0.2496
sportschannel - Similitud: 0.2081
players - Similitud: 0.2012
league - Similitud: 0.1924
***********************************

******* Palabra: computer *******
shopper - Similitud: 0.1349
verlag - Similitud: 0.1248
delicate - Similitud: 0.1196
drive - Similitud: 0.1105
hackers - Similitud: 0.1082
***********************************

******* Palabra: religion *******
religious - Similitud: 0.2475
religions - Similitud: 0.2237
crusades - Similitud: 0.1936
christianity - Similitud: 0.1882
categorized - Similitud: 0.1849
***********************************

******* Palabra: space *******
nasa - Similitud: 0.3178
shuttle - Similitud: 0.2784
exploration - Similitud: 0.2328
aeronautics - Similitud: 0.2219
cfa - Similitud: 0.2164
***********************************

******* Palabra: science *******
behaviorists - Similitud: 0.3941
cognitivists - Similitud: 0.3941
scientific - Similitud: 0.3624
empirical - Similit

## Conclusiones del Estudio de Similitud entre Palabras

El análisis de similitud coseno aplicado a la **matriz término-documento** permite identificar palabras que frecuentemente co-ocurren en los mismos documentos. A continuación, se presentan las conclusiones para cada palabra seleccionada:

---

### Palabra: `hockey`
- Palabras más similares:
  - `ncaa` (0.2672)
  - `nhl` (0.2496)
  - `sportschannel` (0.2081)
  - `players` (0.2012)
  - `league` (0.1924)
- **Conclusión:**
  El modelo identifica palabras relacionadas al ámbito del hockey (`nhl`, `players`, `league`). Sin embargo, algunas palabras como `sportschannel` reflejan conceptos relacionados con medios de transmisión.

---

### Palabra: `computer`
- Palabras más similares:
  - `shopper` (0.1349)
  - `verlag` (0.1248)
  - `delicate` (0.1196)
  - `drive` (0.1105)
  - `hackers` (0.1082)
- **Conclusión:**
  La similitud detecta términos relacionados con tecnología, aunque algunos como `verlag` (editoriales) o `delicate` parecen poco relevantes. Esto puede deberse a asociaciones en documentos no puramente técnicos.

---

### Palabra: `religion`
- Palabras más similares:
  - `religious` (0.2475)
  - `religions` (0.2237)
  - `crusades` (0.1936)
  - `christianity` (0.1882)
  - `categorized` (0.1849)
- **Conclusión:**
  Las palabras relacionadas a `religion` están correctamente identificadas, especialmente aquellas vinculadas a conceptos amplios como `religious`, `christianity` o `crusades`.

---

### Palabra: `space`
- Palabras más similares:
  - `nasa` (0.3178)
  - `shuttle` (0.2784)
  - `exploration` (0.2328)
  - `aeronautics` (0.2219)
  - `cfa` (0.2164)
- **Conclusión:**
  La similitud coseno identifica claramente términos relacionados con la exploración espacial (`nasa`, `shuttle`, `exploration`). Los resultados son coherentes con un tema técnico y específico.

---

### Palabra: `science`
- Palabras más similares:
  - `behaviorists` (0.3941)
  - `cognitivists` (0.3941)
  - `scientific` (0.3624)
  - `empirical` (0.2890)
  - `sects` (0.2538)
- **Conclusión:**
  Este término está asociado tanto con ciencias duras (`scientific`, `empirical`) como con teorías relacionadas a psicología o filosofía (`behaviorists`, `cognitivists`). El término `sects` podría deberse a documentos que relacionan ciencia con religión o pseudociencias.

---

## Observaciones Generales
1. Las palabras seleccionadas muestran asociaciones coherentes con sus contextos temáticos.
2. Algunas palabras irrelevantes (`verlag`, `delicate`) pueden aparecer si se relacionan con documentos técnicos por coincidencias superficiales.
3. Este método es efectivo para identificar palabras relacionadas dentro de un corpus extenso, pero puede ser sensible al ruido generado por documentos con contenido ambiguo.

